In [6]:
library(tidyverse)
library(repr)
library(readxl)
source("cleanup.R")
options(repr.matrix.max.rows = 6)

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.5
✔ forcats   1.0.0     ✔ stringr   1.5.1
✔ ggplot2   3.5.1     ✔ tibble    3.2.1
✔ lubridate 1.9.3     ✔ tidyr     1.3.1
✔ purrr     1.0.2     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors
Warning message in file(filename, "r", encoding = encoding):
“cannot open file 'cleanup.R': No such file or directory”


ERROR: Error in file(filename, "r", encoding = encoding): cannot open the connection


## Data Description 
The player.csv data contains information for 196 players, each observation contians 6 variables:
- **experience (character)**: there are five experience level of players
   - Beginner
   - Amateur
   - Regular
   - Veteran
   - Pro
     <br><br>
- **subscription(logical)**: subscribed to a game newsletter or not
  <br><br>
- **hashedEmail (character)**：email of the play that has been processed to protect privacy, likely do not have meaningful information to analyse.
  <br><br>
- **played_hours(decimal)**: played time
  - min:0.0 hours
  - 1st Qu.:0.00 hours
  - median: 0.10 hours
  - mean: 5.85 hours
  - 3rd Qu.: 0.60 hours
  - Max.:223.10 hours
    <br><br>
- **name(character)**
  <br><br>
- **gender(character)**
  <br><br>
- **Age(integer)**
  - Min.: 9.0
  - 1st Qu.:17.00
  - median: 19.00
  - mean: 22.75
  - 3rd Qu.: 58.00
  - Max.:58.00
  - Na's:2

In [15]:
players <- read.csv("players.csv")
played_hours_statistic <- players |>
                        select(played_hours)|>
                        summary()

age_statistic <- players |>
                select(Age)|>
                summary()

played_hours_statistic
age_statistic

  played_hours    
 Min.   :  0.000  
 1st Qu.:  0.000  
 Median :  0.100  
 Mean   :  5.846  
 3rd Qu.:  0.600  
 Max.   :223.100  

      Age       
 Min.   : 9.00  
 1st Qu.:17.00  
 Median :19.00  
 Mean   :21.14  
 3rd Qu.:22.75  
 Max.   :58.00  
 NA's   :2      

## Questions
**Broad Question:** what player charateristics are associated with subscribing a game newsletter, and how do these features differ between various player types?
<br>
**Specific Question**: can age and play time predict subscribtion to game newsletter?<br>


experience,subscribe,hashedEmail,played_hours,name,gender,Age
<chr>,<lgl>,<chr>,<dbl>,<chr>,<chr>,<int>
Beginner,TRUE,4bed6f49b672974f706aa7f020b4f29020326bf594520593557cd381a51559b8,1.5,Sarah,Female,25
Beginner,TRUE,9db37a1d381697f6c83e3b97b276e38bb9d9c811ba4596edb75d4dbfea6415b0,0.0,Elodie,Female,17
Beginner,TRUE,f9ac013b2f0bc2bd4928a6a0fc8a0aae8b5c4f2670cf9e18f547b7e72034c965,1.0,Leah,Male,17
Beginner,TRUE,89c8e07d6a2aa102d9cd05086d51b2e41195c9c807dd976a06f4262ff36d0cca,0.0,Ren,Male,17
Beginner,TRUE,f459ce1862ec9a43b6c714810390dfd632ed104eb11378b3a2a134b1b107bc75,0.6,Jabari,Male,26
Beginner,FALSE,fa7d496b2f74c51ec70395bd8397b49f97a3ce8d7ba7e08887d82ec0008137a7,0.0,Umar,Male,24
Beginner,TRUE,d13020f8d53c08c95e64f2254424a65c9c479fcfd866b21b850d200a2689a8b5,0.4,Omkar,Prefer not to say,22
Beginner,FALSE,5e5c25a773be7a62638a163d773534e575a5ad5782104723e53cb676a30fa60b,0.0,Jude,Female,42
Beginner,FALSE,1a2b92f18f36b0b59b41d648d10a9b8b20a2adff550ddbcb8cec2f47d4d881d0,0.2,Aurora,Female,37
